## ДЗ Лапшиной Анны АТМ-22-01
### Для исследования влияния некоторых технологических факторов на прочность приклеивания низа обуви полиуретановым клеем были поставлены эксперименты по плану ПФЭ 23, причем каждый эксперимент повторялся по три раза. В качестве факторов, влияющих на прочность y (кг/2,5 см2), были выбраны следующие: 
### z1 – количество наносимого клея (г/см2), z1 = 0,02...0,06; 
### z2 – время активации клеевой пленки (сек), z2 = 60...300;
### z3 – давление прессования при склеивании (кгс/см2), z3 = 2...8. 
### Требуется построить уравнение регрессии, учитывая все взаимодействия факторов, проверить полученную модель на адекватность и произвести ее интерпретацию.


In [1]:
import scipy.stats as stats
import numpy as np
import pandas as pd

# Функция нахождения центра фактора
def center (max, min):
    z0 = (max-min)/2
    return z0

# Функция нахождения средних выборочных результатов для каждого эксперимента: 
def y_average (y1, y2, y3):
    y_aver = np.zeros(8)
    for i in range(0,8):
        y_aver[i] = round((y1[i]+y2[i]+y3[i])/3, 2)
    return y_aver

# Функция нахождения коэффициентов парных взаимодействий в матрице планирования
def pair_interactions(x1, x2):
    x12 = np.zeros(8)
    for i in range(0,8):
        x12[i] = x1[i]*x2[i]
    return x12

# Функция нахождения коэффициентов регрессии 
def regression_coeff(n, y, x):
    sum = 0
    for i in range(0,8):
        sum += y[i]*x[i]
    b = (1/n)*sum
    return round(b, 2)

# Функция нахождения суммы выборочных дисперсий
def select_var(y_aver, y1, y2, y3, m):
    sj2 = np.zeros(8)
    for i in range(0,8):
        sj2[i] = (1/(m-1))*(((y1[i]-y_aver[i])**2)+((y2[i]-y_aver[i])**2)+((y3[i]-y_aver[i])**2))
    return sj2

# Функция нахождения дисперсии воспроизводимости
def var_of_reproducibility(sj2, n):
    sy2 = sum(sj2)/n
    return round(sy2,2)

# Функция нахождения остаточной дисперсии
def residual_var(b0,b1,b2,b3,b12,b13,b23,b123,x1,x2,x3,x12,x13,x23,x123,y_aver,n,m):
    y = np.zeros(8)
    sum = 0
    for i in range(0,8):
        y[i] = b0 + b1*x1[i] + b2*x2[i] + b3*x3[i] + b12*x12[i] + b13*x13[i] + b23*x23[i] + b123*x123[i]
        sum += (y[i]-y_aver[i])**2
    s_ost2 = (m/(n-7))*sum
    return round(s_ost2,3)


In [2]:
# Исходные данные
n = 8
m = 3
alpha = 0.05
y1 = np.array([7.4, 8.6, 12.3, 5.8, 18.8, 8.4, 11.8, 10.5]) 
y2 = np.array([8.4, 7, 9, 5.8, 17, 8.4, 7, 7.8]) 
y3 = np.array([6.4, 7.8, 9.3, 5.7, 15.2, 6, 9.4, 8.1]) 
z1_max = 0.06
z1_min = 0.02
z2_max = 300
z2_min = 60
z3_max = 8
z3_min = 2


# Находим центры факторов, составляем матрицу чередующихся коэффициентов и определяем коэф. парного взаимодействия
z10 = center(z1_max, z1_min)
z20 = center(z2_max, z2_min)
z30 = center(z3_max, z3_min)
x0 = np.ones(8)
x1 = np.array([1, -1, 1, -1, 1, -1, 1, -1])
x2 = np.array([1, 1, -1, -1, 1, 1, -1, -1])
x3 = np.array([1, 1, 1, 1, -1, -1, -1, -1])
x12 = pair_interactions(x1, x2)
x13 = pair_interactions(x1, x3)
x23 = pair_interactions(x2, x3)
x123 = pair_interactions(x12, x3)

# Считаем среднее выборочное результатов для каждого эксперимента
y_aver = y_average(y1,y2,y3)
print(f'{y_aver} - среднее выборочное результатов')

# Определяем коэффициенты регрессионного уравнения
b0 = regression_coeff(n, y_aver, x0)
b1 = regression_coeff(n, y_aver, x1)
b2 = regression_coeff(n, y_aver, x2)
b3 = regression_coeff(n, y_aver, x3)
b12 = regression_coeff(n, y_aver, x12)
b13 = regression_coeff(n, y_aver, x13)
b23 = regression_coeff(n, y_aver, x23)
b123 = regression_coeff(n, y_aver, x123)
print(f'{b0, b1, b2, b3, b12, b13, b23, b123} - коэффициенты регрессионного уравнения')

# Считаем выборочную дисперсию и дисперсию воспроизводимости
sj2 = select_var(y_aver, y1, y2, y3, m)
sy2 = var_of_reproducibility(sj2, n)
print(f'{sy2} - дисперсия воспроизводимости')

# Находим среднее квадратичное отклонение коэффициентов
sko = round((sy2/(n*m))**(1/2), 3)
print(f'{sko} - среднее квадратичное отклонение коэффициентов')

# Определяем критическое значение распределения Стьюдента при уровне значимости 0.05 и сравниваем граничное значение с найденными коэфф-и
t_kr = round(stats.t.ppf(1-alpha/2, df= n*(m-1)), 2)
print(f'{t_kr} - критическое значение распределения Стьюдента при уровне значимости 0.05')
limit_coeff = round(t_kr*sko,2)
print(f'{limit_coeff} - граничное значение')
# Обнуляем коэф 12 по результатам сравнения
b12 = 0

# Рассчитываем остаточную дисперсию для нахождения значения критерия Фишера
s_ost2 = residual_var(b0,b1,b2,b3,b12,b13,b23,b123,x1,x2,x3,x12,x13,x23,x123,y_aver,n,m)
print(f'{s_ost2} - остаточная дисперсия')
fisher_calc = s_ost2/sy2
# Определяем расчетное значение критерия Фишера и сравниваем 
fisher_table = round(stats.f.ppf(1-alpha, 1, 16),2)
print(f'{round(fisher_calc,2)} - расчетное значение критерия Фишера')
print(f'{fisher_table} - табличное значение критерия Фишера')
print(f'{fisher_table} > {round(fisher_calc,2)}, поэтому делаем вывод об адекватности уравнения регрессии')




[ 7.4   7.8  10.2   5.77 17.    7.6   9.4   8.8 ] - среднее выборочное результатов
(9.25, 1.75, 0.7, -1.45, 0.5, -0.75, -0.9, -1.7) - коэффициенты регрессионного уравнения
2.26 - дисперсия воспроизводимости
0.307 - среднее квадратичное отклонение коэффициентов
2.12 - критическое значение распределния Стьюдента при уровне значимости 0.05
0.65 - граничное значение
5.913 - остаточная дисперсия
2.62 - расчетное значение критерия Фишера
4.49 - табличное значение критерия Фишера
4.49 > 2.62, поэтому делаем вывод об адекватности уравнения регрессии


In [3]:
# Визуализируем таблицу с помощью Pandas
print('Матрица планирования для обработки результатов')
data1 = {'x1' : x1,
        'x2' : x2,
        'x3' : x3,
        'x12' : x12,
        'x13' : x3,
        'x23' : x23,
        'x123' : x123,
        'y1' : y1,
        'y2' : y2,
        'y3' : y3,
        'y*' : y_aver}
frame1 = pd.DataFrame(data1)
print(frame1)
print('')

Матрица планирования для обработки результатов
   x1  x2  x3  x12  x13  x23  x123    y1    y2    y3     y*
0   1   1   1  1.0    1  1.0   1.0   7.4   8.4   6.4   7.40
1  -1   1   1 -1.0    1  1.0  -1.0   8.6   7.0   7.8   7.80
2   1  -1   1 -1.0    1 -1.0  -1.0  12.3   9.0   9.3  10.20
3  -1  -1   1  1.0    1 -1.0   1.0   5.8   5.8   5.7   5.77
4   1   1  -1  1.0   -1 -1.0  -1.0  18.8  17.0  15.2  17.00
5  -1   1  -1 -1.0   -1 -1.0   1.0   8.4   8.4   6.0   7.60
6   1  -1  -1 -1.0   -1  1.0   1.0  11.8   7.0   9.4   9.40
7  -1  -1  -1  1.0   -1  1.0  -1.0  10.5   7.8   8.1   8.80

